# And Then There Were None KG

In [1]:
import rdflib
import xlrd
import pandas
from rdflib import URIRef, Literal, BNode,Namespace

data = { # similar format for how you made your pandas data fram
    "subject": [],
    "predicate": [],
    "object" : []
}
wb = xlrd.open_workbook("ATTWN triples.xlsx")
sheet = wb.sheet_by_index(0)
for i in range(1, sheet.nrows): # skip the first header line
       
    s = sheet.cell_value(i, 0)
    p = sheet.cell_value(i, 1)
    o = sheet.cell_value(i, 2)
    
    data["subject"].append(s)
    data["predicate"].append(p)
    #print(o)
    data["object"].append(o) 
  
data_processed = {
    'subject':[],'predicate':[],'object': []
}
ugly_token = {
    ' ': '_',
    '"': '',
}
# replace all ugly tokens and copy to new data structure
for x in data.keys(): #x is subj,obj,pred
    for item in data[x]:
        if type(item)!=str:
            data_processed[x].append(item)
            continue
        new_token=item       
        for k in ugly_token:
            new_token=new_token.replace(k, ugly_token[k]) 
            
        data_processed[x].append(new_token)            

n = Namespace("http://UCLA_REU_2020.org/ATTWN/")

g = rdflib.Graph()

for i in range(len(data['subject'])):
    s = n[data_processed['subject'][i]]
    p = n[data_processed['predicate'][i]]
    o_data=data_processed['object'][i]
    if type(o_data)==float or type(o_data)==int:
        o_node=Literal(o_data)
    else:
        o_node=n[o_data]
    g.add((s, p, o_node))

#check g
for s, p, o in g:
   print((s, p, o))

(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/Edward_Armstrong'), rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/found_body_of'), rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/Anthony_Marston'))
(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/Thomas_Rogers'), rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/has_lastname'), rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/Rogers'))
(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/Philip_Lombard'), rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/came_to_island_for'), rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/job'))
(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/Fourth_Soldier'), rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/character_of'), rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/poem'))
(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/boy'), rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/has_predicate'), rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/had_killed'))
(rdf

In [3]:
import matplotlib.pyplot as plt
import networkx as nx
%matplotlib qt
#%matplotlib inline
plt.figure(figsize=(20,20))

edgelabels={}
G = nx.DiGraph()
#plt.clf()
for i in range(len(data['object'])):
    v1 = data['subject'][i]
    v2 = data['object'][i]
    G.add_edge(v1,v2)
    e_lbl = data['predicate'][i]
    edgelabels[(v1, v2)] = e_lbl

#print(edge_labels)

pos = nx.spring_layout(G,k=0.15,iterations=20, scale=3)
nx.draw_networkx(G, pos=pos,font_size=8,node_color='pink')
nx.draw_networkx_edge_labels(G, pos=pos, edge_labels=edgelabels, font_size=7)

plt.show()

In [4]:
qres = g.query(
    """PREFIX foaf: <http://UCLA_REU_2020.org/ATTWN/>
       SELECT ?Firstname
       WHERE {
          ?aname foaf:has_profession foaf:doctor.
          ?aname foaf:has_firstname ?Firstname.
       }""")

for row in qres:
    print(row)

(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/Edward_'),)


In [30]:

def stripURI(x):
    return x.split("/")[-1]
def random_walk_nondir(g, walk_length):
    '''
        Performs random walk over rdflib graph. Does not pay attention to direction of the knowledge graph.
        Usually in KG it is directed as subj-pred-obj
        Returns 2-tuple of lists (nodes_traversed, edges_traversed)
        g::rdflib graph
        walk_length::positive_int
    '''
    subjs = list(g.subjects())
    currentNode = random.choice(subjs)
    nodes_traversed = [stripURI(currentNode)]
    edges_traversed = []
    for i in range(walk_length):
        # get the possible nodes
        preds_objs = list(g.predicate_objects(subject = currentNode)) # (edge, node)
        subj_preds = list(g.subject_predicates(object = currentNode)) # (node, edge)
        # combine the two lists
        # list of (node, edge) tuples
        nodes_edges = [(t[0], t[1]) for t in subj_preds] + [(t[1], t[0]) for t in preds_objs]
        if len(nodes_edges) == 0: # nowhere to go, end walk
            break
        node, edge = random.choice(nodes_edges)
        nodes_traversed.append(stripURI(node))
        edges_traversed.append(stripURI(edge))
        currentNode = node
    return (nodes_traversed, edges_traversed)


In [55]:
cleaned_data = []
for i in range(1000):
    rw_nodes, rw_edges = random_walk_nondir(g, 50)
    lmao = ' '.join(word for word in rw_nodes)
    cleaned_data.append(lmao)

In [65]:
#countvector + LDA
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline
count_vectorizer = CountVectorizer(stop_words='english')
count_data = count_vectorizer.fit_transform(cleaned_data)
import warnings
warnings.simplefilter("ignore", DeprecationWarning)

# Load the LDA model from sk-learn
from sklearn.decomposition import LatentDirichletAllocation as LDA
 
# Helper function
def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
        
number_topics = 7
number_words = 10

# Create and fit the LDA model
lda = LDA(n_components=number_topics)
lda.fit(count_data)

# Print the topics found by the LDA model
print("Topics found via LDA:")
print_topics(lda, count_vectorizer, number_words)


Topics found via LDA:

Topic #0:
ninth_solider frizzled_up philip_lombard_ veteran owen mr doctor religion lieutenant_ inheritance

Topic #1:
past_crime had_killed edward_armstrong emily_brent william_blore vera_claythorne lawrence_wargrave record john_macarthur thomas_rogers

Topic #2:
died_by edward_armstrong past_crime lawrence_wargrave drowning hanging sleep ethel_rogers anthony_martson emily_brent

Topic #3:
poem third_soldier eighth_soldier d_stay_there big_bear_hugged_one edward_armstrong tenth_soldier ethel_rogers hanging john_macarthur

Topic #4:
died_by poem choking first_soldier hatchetwound second_soldier hatchet blow_to_head poison injection

Topic #5:
vera_claythorne philip_lombard edward_armstrong emily_brent william_blore lawrence_wargrave died_by past_crime thomas_rogers ethel_rogers

Topic #6:
poem past_crime ethel_rogers thomas_rogers john_macarthur seventh_soldier fourth_soldier edward_armstrong sixth_soldier fifth_soldier


In [64]:
#TF-idf with LDA
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

count_vectorizer = TfidfVectorizer(stop_words='english')
count_data = count_vectorizer.fit_transform(cleaned_data)
idx_to_word = np.array(count_vectorizer.get_feature_names())
import warnings
warnings.simplefilter("ignore", DeprecationWarning)

# Load the LDA model from sk-learn
from sklearn.decomposition import LatentDirichletAllocation as LDA
 
# Helper function
def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
        
number_topics = 7
number_words = 10

# Create and fit the LDA model
lda = LDA(n_components=number_topics)
lda.fit(count_data)

# Print the topics found by the LDA model
print("Topics found via LDA:")
print_topics(lda, count_vectorizer, number_words)


Topics found via LDA:

Topic #0:
hatchetwound hatchet philip_lombard_ veteran died_by d_stay_there ethel job housekeeper injection

Topic #1:
choking first_soldier second_soldier poison ten_little_soldiers poem overslept died_by drowning crushed

Topic #2:
past_crime had_killed edward_armstrong vera_claythorne poem philip_lombard william_blore john_macarthur emily_brent thomas_rogers

Topic #3:
poem fourth_soldier fifth_soldier eighth_soldier chopped_in_halves big_bear_hugged_one bee_sting edward_armstrong past_crime emily_brent

Topic #4:
blow_to_head seventh_soldier rock red_herring_swallowed_one died_by poem john_macarthur edward_armstrong anthony_martson philip_lombard

Topic #5:
hanging died_by noose tenth_soldier gunshot gun sleep pills past_crime syringe

Topic #6:
died_by past_crime edward_armstrong vera_claythorne philip_lombard lawrence_wargrave emily_brent william_blore ethel_rogers thomas_rogers


In [63]:
#Tf-idf with NMF
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

count_vectorizer = TfidfVectorizer(stop_words='english')
count_data = count_vectorizer.fit_transform(cleaned_data)
idx_to_word = np.array(count_vectorizer.get_feature_names())
from sklearn.decomposition import NMF
nmf = NMF(n_components=5, solver="mu")
H = nmf.fit_transform(count_data)
W = nmf.components_
 
# print the topics
 
for i, topic in enumerate(W):
 
    print("Topic {}: {}".format(i + 1, ",".join([str(x) for x in idx_to_word[topic.argsort()[-10:]]])))

Topic 1: john_macarthur,thomas_rogers,ethel_rogers,lawrence_wargrave,emily_brent,william_blore,philip_lombard,vera_claythorne,edward_armstrong,past_crime
Topic 2: lieutenant,twentyone_people,patient,boy,maid,employer,two_children,suspect,past_crime,had_killed
Topic 3: crushed,injection,hatchetwound,gunshot,shove,blow_to_head,hanging,sleep,drowning,died_by
Topic 4: bee_sting,red_herring_swallowed_one,d_stay_there,sixth_soldier,eighth_soldier,fourth_soldier,fifth_soldier,seventh_soldier,third_soldier,poem
Topic 5: retired_judge,macarthur,past_crime,emily_brent,overslept,ten_little_soldiers,poison,second_soldier,first_soldier,choking
